# Sentence Classification

## Importing necessary libraries

In [225]:
#general
import pandas as pd
import numpy as np
import warnings
import re

# NLP
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

#ML
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn import metrics 
from sklearn.model_selection import  train_test_split
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

warnings.filterwarnings("ignore")

In [240]:
# Reading train/test Data
train = pd.read_csv('../dataset/train.csv')
test = pd.read_csv('../dataset/test.csv')

In [227]:
train.head()

,Sentence_id,New_Sentence,Type
0,GERRES15609,<html> b'Author and/or Review architecture/des...,Responsibility
1,PHERES15784,<html> b'Should be able to develop custom dyna...,Responsibility
2,GERREQ10457,<html> b'Experience in working cross\\u2010fun...,Requirement
3,GERSKL27235,"<html> b'Previous business experience, includi...",Skill
4,HONSSK18415,b'Delivering fast and right the first \\U0001f...,SoftSkill


In [228]:
test.head()

,Sentence_id,New_Sentence
0,UAERES16346,<html> b'Collaborate across all of DB&T practi...
1,COGREQ15586,b'Strong \\u2705 technology expertise in Ident...
2,UAEREQ12722,b'Strong knowledge <br> on Service \\U0001f517...
3,COGSKL29155,<html> b'Architect scalable data processing an...
4,PHERES12551,"<html> b'Map client organization, build outsta..."


In [229]:
#checking null values
train.isna().sum()

Sentence_id     0
New_Sentence    0
Type            0
dtype: int64

#### report:
- No null values

In [230]:
import re
CLEANTAGS = re.compile('<.*?>')
CLEANB= re.compile('b\'')
CLEANC = re.compile('\\.*?')
def clean(data):
    cleantag = re.sub(CLEANTAGS, '', data)
    cleanb = re.sub(CLEANB, '', cleantag)
    cleanc = re.sub(CLEANC, '', cleanb)
    return cleanc

In [231]:
for i in range(len(train)):
    train.iloc[i,1]=clean(train.iloc[i,1])

In [232]:
for i in range(len(test)):
    test.iloc[i,1]=clean(test.iloc[i,1])

In [233]:
lemmatizer = WordNetLemmatizer()
train_corpus = []

for i in range(len(train)):
    sent = re.sub('[^a-zA-Z]', ' ', train['New_Sentence'][i])
    sent = sent.lower()
    sent = nltk.word_tokenize(sent)
    sent = [lemmatizer.lemmatize(word) for word in sent if not word in set(
        stopwords.words('english'))]
    sent = ' '.join(sent)
    train_corpus.append(sent)

In [234]:
lemmatizer = WordNetLemmatizer()
test_corpus = []

for i in range(len(test)):
    sent = re.sub('[^a-zA-Z]', ' ', train['New_Sentence'][i])
    sent = sent.lower()
    sent = nltk.word_tokenize(sent)
    sent = [lemmatizer.lemmatize(word) for word in sent if not word in set(
        stopwords.words('english'))]
    sent = ' '.join(sent)
    test_corpus.append(sent)

### Experiment 1: Bag of Word 

In [235]:
cv = CountVectorizer(max_features=7000)
X_BOW = cv.fit_transform(train_corpus)

In [236]:
cv = CountVectorizer(max_features=7000)
X_test = cv.fit_transform(test_corpus)

In [241]:
le = LabelEncoder()
train['Type'] = le.fit_transform(train['Type'])
y = train['Type']
le.classes_

array(['Education', 'Experience', 'Requirement', 'Responsibility',
       'Skill', 'SoftSkill'], dtype=object)

In [218]:
def evaluate_clf(true, predicted):
    acc = accuracy_score(true, predicted) 
    return acc

In [187]:
# Create a function which can evaluate models and return a report 
def evaluate_models(X, y, models):
    # separate dataset into train and test
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
    
    cost_list=[]
    models_list = []
    accuracy_list = []
    
    for i in range(len(list(models))):
        model = list(models.values())[i]
        model.fit(X_train, y_train) 

        # Make predictions
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        # Training set performance
        model_train_accuracy=evaluate_clf(y_train ,y_train_pred)
        
        # Test set performance
        model_test_accuracy=evaluate_clf(y_test, y_test_pred)

        print(list(models.keys())[i])
        models_list.append(list(models.keys())[i])
        print('Model performance for Training set')
        print("- Accuracy: {:.4f}".format(model_train_accuracy))

        print('-'*35)

        print('Model performance for Test set')
        print('Accuracy: {:.4f}'.format(model_test_accuracy))
        print('='*35)
        print('\n')

In [193]:
# Dictionary which contains models for experiment
models = {
    "Naive bais": MultinomialNB(),
    "Random Forest": RandomForestClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "Logistic Regression": LogisticRegression(),
     "K-Neighbors Classifier": KNeighborsClassifier(),
    "XGBClassifier": XGBClassifier(), 
     "CatBoosting Classifier": CatBoostClassifier(verbose=False),
    "AdaBoost Classifier": AdaBoostClassifier()
}

In [194]:
evaluate_models(X_BOW, y, models)

Naive bais
Model performance for Training set
- Accuracy: 0.7058
-----------------------------------
Model performance for Test set
- Accuracy: 0.6718


Random Forest
Model performance for Training set
- Accuracy: 0.9684
-----------------------------------
Model performance for Test set
- Accuracy: 0.7245


Decision Tree
Model performance for Training set
- Accuracy: 0.9684
-----------------------------------
Model performance for Test set
- Accuracy: 0.6643


Gradient Boosting
Model performance for Training set
- Accuracy: 0.7012
-----------------------------------
Model performance for Test set
- Accuracy: 0.6835


Logistic Regression
Model performance for Training set
- Accuracy: 0.8190
-----------------------------------
Model performance for Test set
- Accuracy: 0.7291


K-Neighbors Classifier
Model performance for Training set
- Accuracy: 0.6868
-----------------------------------
Model performance for Test set
- Accuracy: 0.5371


XGBClassifier
Model performance for Training set

### Experiment 2: TF-IDF

In [195]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=7000)
X_TF_IDF = tfidf.fit_transform(train_corpus)

In [196]:
evaluate_models(X_TF_IDF, y, models)

Naive bais
Model performance for Training set
- Accuracy: 0.7140
-----------------------------------
Model performance for Test set
- Accuracy: 0.6766


Random Forest
Model performance for Training set
- Accuracy: 0.9684
-----------------------------------
Model performance for Test set
- Accuracy: 0.7327


Decision Tree
Model performance for Training set
- Accuracy: 0.9684
-----------------------------------
Model performance for Test set
- Accuracy: 0.6762


Gradient Boosting
Model performance for Training set
- Accuracy: 0.7145
-----------------------------------
Model performance for Test set
- Accuracy: 0.6848


Logistic Regression
Model performance for Training set
- Accuracy: 0.7903
-----------------------------------
Model performance for Test set
- Accuracy: 0.7378


K-Neighbors Classifier
Model performance for Training set
- Accuracy: 0.6473
-----------------------------------
Model performance for Test set
- Accuracy: 0.4595


XGBClassifier
Model performance for Training set

### Final Report
- we can see, Logistic Regression is best algorithm with test accuracy 73.78%

## Hypertuning Final model

In [197]:
# define models and parameters
model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]

# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X_TF_IDF, y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array w

/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array w

...

/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array w

/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array w

/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array w

/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/viketan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array w

Best: 0.742843 using {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
0.709579 (0.004389) with: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.730799 (0.005461) with: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.720735 (0.004170) with: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
0.732075 (0.004748) with: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
0.736710 (0.004646) with: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
0.739605 (0.004615) with: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
0.742638 (0.004234) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
0.741889 (0.004359) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}
0.742843 (0.003992) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
0.716687 (0.004304) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.716660 (0.004338) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.707888 (0.004145) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.632050 (0.004493) wit

In [198]:
model = grid_result.best_estimator_

In [199]:
tfidf = TfidfVectorizer(max_features=7000)
X_test = tfidf.fit_transform(test_corpus)

In [200]:
y_pred = model.predict(X_test)

In [242]:
le.classes_

array(['Education', 'Experience', 'Requirement', 'Responsibility',
       'Skill', 'SoftSkill'], dtype=object)

In [243]:
y_pred=le.inverse_transform(y_pred)

### Saving Result

In [244]:
result = {'Sentence_id':test['Sentence_id'],'Type':y_pred}
result_df = pd.DataFrame(result)
result_df.to_csv('../result/submission.csv',index=False)

In [245]:
result_df.shape

(15029, 2)